In [1]:
import pandas as pd
import numpy as np
import math

# Function to calculate Euclidean distance between two points
def euclidean_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Load the CSV file into a DataFrame
input_file = "pennines_merged_JN_mchi.csv"
df = pd.read_csv(input_file)

# Print out the column names for inspection
print("Columns in the CSV:", df.columns)

# Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Print out the cleaned column names for verification
print("Cleaned Columns in the CSV:", df.columns)

# Initialize a new column in the DataFrame for sinuosity results
df['sinuosity'] = np.nan

# Loop through each junction in the DataFrame
for index, row in df.iterrows():
    junction = row['junction']
    receiver_junction = row['receiver_junction']
    
    # Get the coordinates and flow distance of the current junction
    x1, y1, flow_distance1 = row['x'], row['y'], row['flow_distance']
    
    # Find the receiver junction data
    receiver_row = df[df['junction'] == receiver_junction]
    if receiver_row.empty:
        continue  # If receiver junction is not found, skip this row

    # extract the receiver's coordinates and flow distance
    receiver_data = receiver_row.iloc[0]  # Ensure indexing is safe
    x2, y2, flow_distance2 = receiver_row.iloc[0]['x'], receiver_row.iloc[0]['y'], receiver_row.iloc[0]['flow_distance']
    
    # Calculate Euclidean distance
    euclidean_dist = euclidean_distance(x1, y1, x2, y2)
    
    # Calculate flow distance difference
    flow_distance_diff = flow_distance1 - flow_distance2
    
    # Calculate sinuosity (flow distance difference divided by Euclidean distance)
    if euclidean_dist != 0:
        sinuosity = flow_distance_diff / euclidean_dist
    else:
        sinuosity = np.nan  # Handle division by zero case
    
    # Add sinuosity value to the new column
    df.at[index, 'sinuosity'] = sinuosity

print(f"Junction {junction}: Flow Distance = {flow_distance1}, Euclidean Distance = {euclidean_dist}")
print(f"Junction {junction}: x1 = {x1}, y1 = {y1}, Receiver {receiver_junction}: x2 = {x2}, y2 = {y2}")

Columns in the CSV: Index(['Unnamed: 0', 'junction', 'node', 'x', 'y', 'latitude', 'longitude',
       ' stream_order', ' receiver_junction', 'geometry', 'node.1', 'row',
       'col', 'latitude.1', 'longitude.1', 'chi', 'elevation', 'flow_distance',
       'drainage_area', 'm_chi', 'b_chi', 'source_key', 'basin_key',
       'segmented_elevation', 'dist'],
      dtype='object')
Cleaned Columns in the CSV: Index(['Unnamed: 0', 'junction', 'node', 'x', 'y', 'latitude', 'longitude',
       'stream_order', 'receiver_junction', 'geometry', 'node.1', 'row', 'col',
       'latitude.1', 'longitude.1', 'chi', 'elevation', 'flow_distance',
       'drainage_area', 'm_chi', 'b_chi', 'source_key', 'basin_key',
       'segmented_elevation', 'dist'],
      dtype='object')
Junction 4400: Flow Distance = 142410.0, Euclidean Distance = 1403.2195124070931
Junction 4400: x1 = 541012.5, y1 = 6024987.5, Receiver 4357: x2 = 542387.5, y2 = 6025267.5


In [2]:
# Count of sinuosity values less than 1, excluding 0.0
sinuosity_less_than_1 = df[(df['sinuosity'] < 1) & (df['sinuosity'] != 0.0)].shape[0]

# Count of sinuosity values greater than or equal to 1, excluding 0.0
sinuosity_greater_equal_1 = df[(df['sinuosity'] >= 1) & (df['sinuosity'] != 0.0)].shape[0]

# Count of sinuosity values equal to 0.0
sinuosity_equal_0 = df[df['sinuosity'] == 0].shape[0]

# Output the results
print(f"Number of junctions with sinuosity < 1 (excluding 0.0): {sinuosity_less_than_1}")
print(f"Number of junctions with sinuosity >= 1 (excluding 0.0): {sinuosity_greater_equal_1}")
print(f"Number of junctions with sinuosity = 0: {sinuosity_equal_0}")


Number of junctions with sinuosity < 1 (excluding 0.0): 3
Number of junctions with sinuosity >= 1 (excluding 0.0): 1375
Number of junctions with sinuosity = 0: 0


In [3]:
# Save the updated DataFrame to csv
output_file = "pennines_sinuosity.csv"
df.to_csv(output_file, index=False)